In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prepara

In [3]:
import pandas as pd
from pandas import DataFrame as pddf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

In [4]:
df = pd.read_csv("/content/drive/MyDrive/ApplyAI Dataset/training_sample.csv").dropna()
datadir = "/content/drive/MyDrive/ApplyAI Dataset/sample dataset/"

In [5]:
df_copy = df.copy()

In [6]:
# drop NA values and make landmark number int
df_copy.dropna(axis = 0)
df_copy = df_copy.astype({'landmark number': 'int'})
df_copy.shape

(262, 4)

## Implementing dataset spliting

### 1 frame of dataset spilting

In [7]:
# 
df_train = df_copy[['landmark number', 'continent']].copy()
df_test = df_copy[['landmark number', 'continent']].copy()
df_val = df_copy[['landmark number', 'continent']].copy()


In [8]:
# append df_train
for i in range(0,2):
  df_train = df_train.append(df_test, ignore_index=True)


In [9]:
df_train = df_train.sort_values(by=['landmark number'])
df_train.head()

,landmark number,continent
0,1,Asia
262,1,Asia
524,1,Asia
1,2,Asia
263,2,Asia


In [10]:
df_train.reset_index(inplace=True, drop = True)
df_train.head()

,landmark number,continent
0,1,Asia
1,1,Asia
2,1,Asia
3,2,Asia
4,2,Asia


### 2 add file name

In [11]:
length_df = len(df_copy)
#print(length_df)  
test = [] #test
train = [] #sss
val = []
for i in range(300):
  if (i >= 155 and i <= 179) or (i >= 222 and i <= 230) or (i >= 254 and i <= 257):
    continue
  rs = []
  rs.append(df_copy["files"][i][0:6])
  rs.append(df_copy["files"][i][6:11])
  rs.append(df_copy["files"][i][12:17])
  rs.append(df_copy["files"][i][18:23])
  rs.append(df_copy["files"][i][24:29])
  random.shuffle(rs)
  
  test.append(rs[3])
  #rs.remove(rs[0])
  val.append(rs[4])
  #rs.remove(rs[0])

  for i in range(3):
    train.append(rs[i])
    #rs.remove(rs[0])
  

  #df_train['files']  

'''
181-222
232-254
259-300
  test = rs[4]
  df_test.loc[i,2] = test
  for j in range(0,4):
    #261
    #train = ls[j]
    print(i + 261*j)
    df_train.loc[i + 261*j, 2] = ls[j]
'''


'\n181-222\n232-254\n259-300\n  test = rs[4]\n  df_test.loc[i,2] = test\n  for j in range(0,4):\n    #261\n    #train = ls[j]\n    print(i + 261*j)\n    df_train.loc[i + 261*j, 2] = ls[j]\n'

### finish build train data

In [12]:
df_tl = pddf(train, columns = ['id'])
df_tl.head()

,id
0,00105
1,00104
2,00101
3,00205
4,00204


In [13]:
frames_train = [df_train, df_tl]
df_tra = pd.concat(frames_train, axis = 1)
df_tra.head()

,landmark number,continent,id
0,1,Asia,00105
1,1,Asia,00104
2,1,Asia,00101
3,2,Asia,00205
4,2,Asia,00204


### finish build test dataset


In [14]:
df_tel = pddf(test, columns = ['id'])
df_tel.head()

,id
0,00102
1,00201
2,00304
3,00402
4,00502


In [15]:
df_tel.shape

(262, 1)

In [16]:
df_test.reset_index(inplace=True, drop = True)

In [17]:
frames_test = [df_test, df_tel]
df_te = pd.concat([df_test, df_tel], axis=1, join='inner')
df_te.shape

(262, 3)

### build validation dataset

In [18]:
df_val_label = pddf(val, columns = ['id'])
df_val_label.head()

,id
0,00103
1,00203
2,00303
3,00401
4,00503


In [19]:
df_val.reset_index(inplace=True, drop = True)
df_valout = pd.concat([df_val, df_val_label], axis=1, join='inner')
df_valout.shape

(262, 3)

### convert to csv file

In [20]:
df_tra.to_csv(path_or_buf = "/content/drive/MyDrive/ApplyAI Dataset/train33.csv",index = False)
df_te.to_csv(path_or_buf = "/content/drive/MyDrive/ApplyAI Dataset/test33.csv",index = False)
df_valout.to_csv(path_or_buf = "/content/drive/MyDrive/ApplyAI Dataset/val33.csv",index = False)

In [21]:
df_tl.to_csv(path_or_buf = "/content/drive/MyDrive/ApplyAI Dataset/train1.csv",index = False)

In [22]:
df_training = pd.read_csv("/content/drive/MyDrive/ApplyAI Dataset/train3.csv")
df_testing = pd.read_csv("/content/drive/MyDrive/ApplyAI Dataset/test3.csv")
df_valing = pd.read_csv("/content/drive/MyDrive/ApplyAI Dataset/test3.csv")

# Start training samples

In [23]:
import pathlib

import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

import PIL.Image
import albumentations.pytorch
import cv2
import matplotlib.pyplot as plt

In [24]:
from tqdm.notebook import tqdm
from typing import List, Tuple

In [25]:
IMAGE_SIZE = 32
BATCH_SIZE = 224

MODEL_FILE = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/model.pth')

TRAIN_LABEL_FILE = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/train3.csv')
TRAIN_IMAGE_DIR = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/sample dataset')

VALID_LABEL_FILE = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/val3.csv')
VALID_IMAGE_DIR = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/sample dataset')

TEST_LABEL_FILE = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/test3.csv')
TEST_IMAGE_DIR = pathlib.Path('/content/drive/MyDrive/ApplyAI Dataset/sample dataset')

In [26]:
pd.read_csv(TRAIN_LABEL_FILE)['id'].values[0:5]

array([103, 101, 102, 205, 203])

In [27]:
def directory(n, image_dir):
  s = str(n)
  if len(s) == 3:
    s = '00' + s
  elif len(s) == 4:
    s = '0' + s
  if n < 16000:
    #ret = image_dir + s + '.jpeg'
    ret = image_dir / f'{s}.jpeg'
  else:
    #ret = image_dir + s + '.jpg'
    ret = image_dir / f'{s}.jpg'
  return ret


# model

In [28]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, label_file: pathlib.Path, image_dir: pathlib.Path) -> None:
        super().__init__()
        # TO CHANGE!!!
        self.files = [
            directory(n, image_dir)
            for n in pd.read_csv(label_file)['id'].values]
        
        self.transformer = albumentations.Compose([
            albumentations.SmallestMaxSize(IMAGE_SIZE, interpolation=cv2.INTER_CUBIC),
            albumentations.CenterCrop(IMAGE_SIZE, IMAGE_SIZE),
            albumentations.Normalize(),
            albumentations.pytorch.ToTensorV2(),
        ])

    def __len__(self) -> int:
        return len(self.files)

    def __getitem__(self, index: int) -> Tuple[str, torch.Tensor]:
        path = self.files[index]
        image = PIL.Image.open(self.files[index])
        image = self.transformer(image=np.array(image))['image']

        return path.name[:-4], image

In [29]:
@torch.no_grad()
def get_features(
    model: nn.Module,
    label_file: pathlib.Path,
    image_dir: pathlib.Path,
) -> Tuple[List[str], torch.Tensor]:
    loader = torch.utils.data.DataLoader(
        Dataset(label_file, image_dir),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = model.cuda()
    model.eval()
    
    all_names = []
    all_features = []

    for names, images in tqdm(loader, desc=image_dir.name):
        images = images.cuda()
        features = model(images)
        all_features.append(features)
        all_names.extend(names)

    return all_names, F.normalize(torch.cat(all_features, dim=0))

In [30]:
def get_similarity(model: nn.Module)-> Tuple[List[str], List[str]]:
    # features
    train_names, train_features = get_features(
        model, TRAIN_LABEL_FILE, TRAIN_IMAGE_DIR)    
    _, valid_features = get_features(
        model, VALID_LABEL_FILE, VALID_IMAGE_DIR)
    test_names, test_features = get_features(
        model, TEST_LABEL_FILE, TEST_IMAGE_DIR)
    print(train_names)
    # penalties
    train_penalties_list = []
    for i in range(0, train_features.shape[0], 128):
        x = torch.mm(train_features[i:i + 128], valid_features.T)
        x = torch.topk(x, k=5)[0].mean(dim=1)
        train_penalties_list.append(x)
    train_penalties = torch.cat(train_penalties_list, dim=0)
    print("train penalties computed")
    
    test_penalties_list = []
    for i in range(0, test_features.shape[0], 128):
        x = torch.mm(test_features[i:i + 128], valid_features.T)
        x = torch.topk(x, k=10)[0].mean(dim=1)
        test_penalties_list.append(x)
    test_penalties = torch.cat(test_penalties_list, dim=0)
    print("test penalties computed")

    # neighbors
    submit_ids = []
    submit_landmark_ids = []
    submit_confidences = []
    
    train_df = pd.read_csv(TRAIN_LABEL_FILE)[['id','landmark number']]
    idmap = {n: v for v, n in train_df.values}

    for i in range(0, test_features.shape[0], 128):
        x = torch.mm(test_features[i:i + 128], train_features.T)
        x -= train_penalties[None, :]
        values, indexes = torch.topk(x, k=3)
        
        submit_ids.extend(test_names[i:i + 128])

        for idxs, vals, penalty in zip(indexes, values, test_penalties[i:i + 128]):
            scores = {}
            for idx, val in zip(idxs, vals):
                print(train_names[idx])
                landmark_id = idmap[train_names[idx]]
                if landmark_id in scores:
                    scores[landmark_id] += float(val)
                else:
                    scores[landmark_id] = float(val)
                    
            landmark_id, confidence = max(
                [(k, v) for k, v in scores.items()], key=lambda x: x[1])
            submit_landmark_ids.append(landmark_id)
            submit_confidences.append(confidence - penalty)

    # standardize confidence values
    max_conf = max(submit_confidences)
    min_conf = min(submit_confidences)
    submit_confidences = [
        (v - min_conf) / (max_conf - min_conf) for v in submit_confidences]
    
    # make values for 'landmark' column
    submit_landmarks = [
        f'{i} {c:.8f}' for i, c in zip(submit_landmark_ids, submit_confidences)]
    
    return submit_ids, submit_landmarks

# Implementation

In [31]:
model = torch.jit.load(str(MODEL_FILE))

In [32]:
print(TEST_IMAGE_DIR)

/content/drive/MyDrive/ApplyAI Dataset/sample dataset


In [33]:
test_names, test_features = get_features(model, TEST_LABEL_FILE, TEST_IMAGE_DIR)

sample dataset:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
submit_ids, submit_landmarks = get_similarity(model)

In [ ]:
submit_df = pd.DataFrame({'id': submit_ids, 'landmarks': submit_landmarks})

In [ ]:
submit_df.to_csv('submission.csv', index=False)

# check the submission

In [ ]:
submit_df = pd.read_csv('submission.csv')
submit_df['landmark_id'] = submit_df['landmarks'].apply(lambda x: int(x.split()[0]))
submit_df['confidence'] = submit_df['landmarks'].apply(lambda x: float(x.split()[1]))
train_df = pd.read_csv(TRAIN_LABEL_FILE)

def get_image(path, name):
    
    img = PIL.Image.open(directory(name, path))
    
    if img.width > img.height:
        img = img.resize((256, round(img.height / img.width * 256)))
        new_img = PIL.Image.new(img.mode, (256, 256), (0, 0, 0))
        new_img.paste(img, (0, (256 - img.height) // 2))
    else:
        img = img.resize((round(img.width / img.height * 256), 256))
        new_img = PIL.Image.new(img.mode, (256, 256), (0, 0, 0))
        new_img.paste(img, ((256 - img.width) // 2, 2))
    return np.array(new_img)

rows = 10
fig = plt.figure(figsize=(15, 4 * rows))
for r in range(rows):
    for c in range(3):
        i = r * 3 + c
        test_name, _, label, conf = submit_df.iloc[i].values
        test_image = get_image(TEST_IMAGE_DIR, test_name)
        train_name = train_df.query(f'landmark_id == {label}').iloc[0]['id']
        train_image = get_image(TRAIN_IMAGE_DIR, train_name)
        image = np.concatenate([test_image, train_image], axis=1)
    
        ax = fig.add_subplot(rows, 3, i + 1)        
        ax.set_title(f'Label={label}, Confidence={conf:.2f}')
        ax.axis('off')
        ax.imshow(image)
fig.tight_layout()